In [1]:
import os
import argparse
import time

import matplotlib as mpl
mpl.use('TkAgg') # sets the backend for matplotlib
import matplotlib.pyplot as plt

from utils.redraw_obstacles import redraw_obstacles, set_non_blocking
from utils.create_draw_params import create_draw_params
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object

import numpy as np

from commonroad.prediction.prediction import Prediction, Occupancy
def occupancy_at_time(self, time_step: int):
    """
    Returns the predicted occupancy of the obstacle at a specific time step.

    :param time_step: discrete time step
    :return: predicted occupancy of the obstacle at time step
    """
    occupancy = None
    if time_step == self.initial_state.time_step:
        shape = self.obstacle_shape.rotate_translate_local(
            self.initial_state.position, self.initial_state.orientation)
        occupancy = Occupancy(time_step, shape)
    elif time_step > self.initial_state.time_step and time_step < (self.initial_state.time_step + len(self._prediction.trajectory.state_list)) and   self._prediction is not None:
        occupancy = self._prediction.occupancy_at_time_step(time_step)
    return occupancy

from commonroad.scenario.obstacle import StaticObstacle, DynamicObstacle, Obstacle
DynamicObstacle.occupancy_at_time = occupancy_at_time

In [2]:
file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/highway-ramp.cr.xml'
scenario, _ = CommonRoadFileReader(file_path).open()

In [3]:
def show_lane(scenario):
    plt.figure(figsize=(250, 100))
    draw_params={'time_begin':0}
    handles = {}
    draw_object(scenario, handles=handles, draw_params=draw_params)
    plt.gca().autoscale()
    plt.gca().set_aspect('equal')
    plt.show()

In [4]:
show_lane(scenario)

KeyboardInterrupt: 

In [9]:
def plt_window(sce, dur, draw_params, size, pos):
    set_non_blocking()  # ensures interactive plotting is activated
    figsize = size
    fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    # ax.set_facecolor("black")
    # ax.set_cmap("binary")
    fig.add_axes(ax)
    fig.patch.set_facecolor('black')
    plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
    plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
    plt.set_cmap("plasma")
    plt.gca().set_aspect('equal')
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    plot_limits = [pos[0] - size[0]/2, pos[0] + size[0]/2, pos[1] +-size[1]/2, pos[1] + size[1]/2]
    draw_params['time_begin'] = dur[0]
    draw_object(sce, handles=handles, draw_params=draw_params, plot_limits=plot_limits)
    # plt.gray()
    fig.canvas.draw()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    data = np.zeros(( (duration[1] - duration[0],) + fig.canvas.get_width_height()[::-1]), dtype='bool')

    t1 = time.time()
    # loop where obstacle positions are updated
    for i in range(dur[0], dur[1]):
        # ...
        # change positions of obstacles
        # ...
        draw_params['time_begin'] = i
        redraw_obstacles(sce, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
        buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
        buffer = buffer[:, : , 0].astype(bool)
        data[(i-dur[0]), :, : ] = buffer
        print('fps:', (i - dur[0])/(time.time()-t1))
    return data

In [10]:
size = [200, 40]
pos = [350, 135]
duration = [0, 100]
inch_in_cm = 10
draw_params = create_draw_params()

origin_data = plt_window(sce=scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 11.523224730553563
fps: 16.738282754313467
fps: 19.805098711392535
fps: 21.99912670741158
fps: 23.356680896466237
fps: 24.95789973728865
fps: 25.94764890226723
fps: 27.06792789507905
fps: 27.86563222039316
fps: 27.97039913947575
fps: 28.405881476501307
fps: 28.725584623269416
fps: 29.26816741797273
fps: 29.745224549631807
fps: 29.87022988107373
fps: 30.03528747268761
fps: 29.85624750860056
fps: 29.61851332838498
fps: 28.88745922895511
fps: 28.135113887301344
fps: 27.53817481599871
fps: 26.635859507670084
fps: 26.06579662683938
fps: 25.476361742368162
fps: 25.079628326995515
fps: 24.56919142524281
fps: 24.467499806630062
fps: 24.791992626609815
fps: 25.01028932282023
fps: 25.285642545767374
fps: 25.45790500782298
fps: 25.733031479879504
fps: 25.896936905652414
fps: 25.998346097358567
fps: 26.089756189803214
fps: 26.206592619282173
fps: 26.160399985165757
fps: 26.011408338349014
fps: 25.85651925401747
fps: 25.805190164899148
fps: 25.796593495560547
fps: 25.674805002192755
f

In [17]:
def plt_data(observe):
    fig = plt.figure()
    plt.gray()
    if len(observe.shape) <=2:
        plt.imshow(observe)
    elif len(observe.shape) > 3:
        for r in observe:
            myobj = plt.imshow(r[0, :, :])
            for rr in r:
                myobj.set_data(rr)
                plt.draw()
                plt.pause(0.01)
    else:
        myobj = plt.imshow(observe[0, :, :])
        for row in observe:
            myobj.set_data(row)
            plt.draw()
            plt.pause(0.01)

In [12]:
from commonroad.scenario.scenario import Scenario

ego_scenario = Scenario(dt=0.1, benchmark_id=scenario.benchmark_id)
ego_scenario.lanelet_network = scenario.lanelet_network

In [73]:
leng = 100
dt = 25

start_goal = np.zeros((leng, 8), dtype=np.float64)

ego_id = 0
set_non_blocking()  # ensures interactive plotting is activated
figsize = size
fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
# ax.set_facecolor("black")
# ax.set_cmap("binary")
fig.add_axes(ax)
fig.patch.set_facecolor('black')
plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
plt.set_cmap("plasma")
plt.gca().set_aspect('equal')
handles = {}  # collects handles of obstacle patches, plotted by matplotlib
draw_object(scenario.lanelet_network, handles=handles, draw_params=draw_params)
fig.canvas.draw()
for handles_i in handles:
    if not handles_i:
        handles.pop()

observed = np.zeros(( (leng, dt,) + fig.canvas.get_width_height()[::-1]), dtype='bool')
data = np.zeros((leng,  dt, 4), dtype=np.float64)

index = 0
while index < leng:
    ego = None
    while(scenario.obstacle_by_id(ego_id) == None):
        ego_id = ego_id + 1
    ego = scenario.obstacle_by_id(ego_id)
    ego_id = ego_id + 1
    print(ego_id)
    size = [400, 30]
    scenario.remove_obstacle(ego)
    traj = ego.prediction.trajectory
    initial_time = traj.initial_time_step
    terminated_time = traj.state_list[-1].time_step
    
    start_time = initial_time + 2* dt
    end_time = start_time + 2 * dt
    
    while end_time < terminated_time - 2* dt:
        t =int( (start_time - initial_time)/2)
        pos = traj.state_list[t].position + [100, 0]
        for i in range(0, dt):
            data[index, i, :] = [traj.state_list[t+i].position[0], traj.state_list[t+i].position[1], traj.state_list[t+i].orientation, traj.state_list[t+i].velocity]
        start_goal[index, 0:4] = data[index, 0, :]
        start_goal[index, 4:8] = data[index, -1, :]


        # ego_scenario.add_objects(ego)
        # ego_data = plt_window(sce=ego_scenario, dur=[start_time, end_time], draw_params=draw_params, size=size, pos=pos)
        # ego_scenario.remove_obstacle(ego)
        # data = plt_window(sce=scenario, dur=[start_time, end_time], draw_params=draw_params, size=size, pos=pos)


        plot_limits = [start_goal[index, 0] - size[0]/2, start_goal[index, 0] + size[0]/2, start_goal[index, 1]+-size[1]/2,start_goal[index, 1]+ size[1]/2]
        print(start_goal[index, :])
        t1 = time.time()
        # loop where obstacle positions are updated
        for i in range(0, dt):
            # ...
            # change positions of obstacles
            # ...
            draw_params['time_begin'] = start_time + 2*i
            redraw_obstacles(scenario, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
            buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
            buffer = buffer[:, : , 0].astype(bool)
            observed[index, i, :, : ] = buffer
#             print('fps:', i/(time.time()-t1), " time:", i)
        index = index + 1
        start_time = start_time + 2 * dt
        end_time = end_time + 2 * dt
    scenario.add_objects(ego)

/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/commonroad/scenario/scenario.py:183: UserWarning: <Scenario/obstacle_by_id> Obstacle with ID 137 is not contained in the scenario.
  % obstacle_id)
/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/commonroad/scenario/scenario.py:183: UserWarning: <Scenario/obstacle_by_id> Obstacle with ID 138 is not contained in the scenario.
  % obstacle_id)


140
[138.81 136.62  -0.    27.52 262.58 136.62  -0.    27.41]
fps: 0.0  time: 0
fps: 8.227174384232589  time: 1
fps: 10.235302156236884  time: 2
fps: 10.978732537777557  time: 3
fps: 11.631208702530659  time: 4
fps: 12.120500366420615  time: 5
fps: 12.276257232461722  time: 6
fps: 12.39795400868951  time: 7
fps: 12.697502195753351  time: 8
fps: 12.733270704907111  time: 9
fps: 12.895686560987503  time: 10
fps: 13.056439382684083  time: 11
fps: 13.184801483942339  time: 12
fps: 12.99372975611058  time: 13
fps: 12.998169152105586  time: 14
fps: 13.027970937575258  time: 15
fps: 12.98358712151845  time: 16
fps: 12.938936738850508  time: 17
fps: 13.017344165965488  time: 18
fps: 13.058884276021528  time: 19
fps: 13.10630303738588  time: 20
fps: 13.15708335860275  time: 21
fps: 13.203574843309417  time: 22
fps: 12.78616137717158  time: 23
fps: 12.856593520417508  time: 24
[267.82 136.62  -0.    27.54 397.01 136.62  -0.    27.85]
fps: 0.0  time: 0
fps: 6.473440598834742  time: 1
fps: 8.24787

KeyboardInterrupt: 

In [28]:
plt_data(observed)